In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import kraft
import numpy as np
import pandas as pd

In [ ]:
function = kraft.information.get_ic

In [ ]:
def simulate(shape, n_category=None):

    if n_category is None:

        vector = np.random.normal(size=shape[1])

        matrix = np.random.normal(size=shape)

    else:

        vector = np.random.randint(0, high=n_category, size=shape[1])

        matrix = np.random.randint(0, high=n_category, size=shape)

    axis_1_labels = pd.Index(
        ("Label {}".format(i) for i in range(shape[1])), name="Axis 1"
    )

    return (
        pd.Series(vector, index=axis_1_labels, name="Vector"),
        pd.DataFrame(
            matrix,
            index=pd.Index(
                ("Label {}".format(i) for i in range(shape[0])), name="Axis 0"
            ),
            columns=axis_1_labels,
        ),
    )

## make

In [ ]:
se, df = simulate((32, 16))

se.iloc[-8::2] = np.nan

kraft.function_heat_map.make(se, df, function, vector_ascending=None)

In [ ]:
se, df = simulate((16, 32))

df.iloc[:, 0] = np.nan

df.iloc[-8:, -8:] = np.nan

kraft.function_heat_map.make(se, df, function)

In [ ]:
df = simulate((16, 8))[1]

for vector in (
    (1, 2, 2, 2, 2, 0, 0, 0),
    (1, 1, 2, 2, 2, 0, 0, 0),
):

    se = pd.Series(vector, index=df.columns)

    for se_ascending in (None, True, False):

        se.name = "se_ascending={}".format(se_ascending)

        kraft.function_heat_map.make(
            se,
            df,
            function,
            se_ascending=se_ascending,
            se_data_type="categorical",
        )

In [ ]:
se = pd.read_csv("data/function_heat_map_se.tsv", sep="	", index_col=0, squeeze=True)

df = pd.read_csv("data/function_heat_map_df.tsv", sep="	", index_col=0)

In [ ]:
for n_row in (1, 2, 3):

    kraft.function_heat_map.make(
        df.iloc[0, :],
        df.iloc[:n_row, :],
        function,
        n_sampling=0,
        n_permutation=0,
        plot_std=3,
    )

In [ ]:
kraft.function_heat_map.make(
    se,
    df,
    function,
    n_job=8,
    n_sampling=10,
    n_permutation=10,
    se_data_type="binary",
)

In [ ]:
for shape in (
    (1, 8),
    (2, 8),
    (8, 8),
    (64, 8),
    (8, 2),
    (8, 4),
    (8, 256),
):

    se, df = simulate(shape)

    kraft.function_heat_map.make(se, df, function, n_extreme=None)

## summarize

In [ ]:
n_column = 16

se, df = simulate((16, n_column))

df_0n = simulate((8, n_column), 4)[1]

df_01 = simulate((4, n_column), 2)[1]

df.drop(df.columns.to_numpy()[-3], axis=1, inplace=True)

df_0n.drop(df_0n.columns.to_numpy()[-2], axis=1, inplace=True)

df_01.drop(df_01.columns.to_numpy()[-1], axis=1, inplace=True)

df_dicts = {
    "Continuous": {"df": df, "data_type": "continuous"},
    "Categorical": {"df": df_0n, "data_type": "categorical"},
    "Binary": {"df": df_01, "data_type": "binary"},
}

scores_dicts = {
    name: kraft.function_heat_map.make(se, df_dict["df"], function, plot=False)
    for name, df_dict in df_dicts.items()
}

for plot_only_shared in (False, True):

    kraft.function_heat_map.summarize(
        se, df_dicts, scores_dicts, plot_only_shared=plot_only_shared
    )